
# 05 — Les Décorateurs en Python (Version **Université**)

> **Objectif pédagogique** : comprendre ce qu’est un décorateur **en profondeur**, savoir **quand** l’utiliser, écrire des décorateurs **propres et testables** (simples, paramétrés, empilés, orientés classes), utiliser les décorateurs **standard** (`lru_cache`, `singledispatch`) et réaliser un **retry** avec backoff.  
> À la fin, tu pourras **expliquer clairement** ces notions en entretien et les **implémenter** sans pièges.

## Plan du cours
1. Motivation : pourquoi les décorateurs ?  
2. Rappel technique : fonctions de première classe, **closures**, `functools.wraps`  
3. Décorateur **simple** (timing / logging)  
4. Décorateur **paramétré** (ex. préfixe de log, seuils)  
5. Décorateurs **empilés** : ordre d’application et d’exécution  
6. Décorateur **orienté classe** (garder de l’état)  
7. Décorateurs **standard** : `lru_cache`, `singledispatch`  
8. **Retry** avec **exponential backoff** (cas réseau/OMS)  
9. Bonnes pratiques, pièges, checklist d’entretien  
10. Exercices guidés + corrigé



## 1) Motivation : pourquoi des décorateurs ?

Un décorateur permet d’**enrichir** le comportement d’une fonction **sans modifier son code**.  
On l’utilise pour des **préoccupations transverses** (*cross-cutting concerns*) :
- **Mesure de performance** (timing), **traces** (logging),
- **Contrôles techniques** (retry, backoff, timeouts),
- **Mise en cache** (mémoïsation),
- **Validation** / **authentification**,
- **Compatibilité** (ex : polymorphisme par type avec `singledispatch`).

> Idée clé : séparer le **métier** (la fonction) des **aspects techniques** (logs, timing, retry).  
> Le code reste **lisible**, **testable**, et chaque préoccupation reste **isolée**.



## 2) Rappel technique : fonctions, closures, `wraps`

- En Python, les fonctions sont des **objets** : on peut les **passer en argument** et **les retourner**.
- Une **closure** est une fonction interne qui **capture** des variables du contexte.
- `functools.wraps` est **indispensable** : il **préserve** le nom et la docstring de la fonction décorée (utile pour les logs, l’aide, et les tests).


In [ ]:

import functools, time

def timing(func):
    '''Décorateur simple qui mesure la durée d'exécution.'''
    @functools.wraps(func)  # ← préserve __name__, __doc__, etc.
    def wrapper(*args, **kwargs):
        t0 = time.perf_counter()
        try:
            return func(*args, **kwargs)
        finally:
            dt = (time.perf_counter() - t0) * 1000
            print(f"{func.__name__} a pris {dt:.2f} ms")
    return wrapper

@timing
def calc(n=80_000):
    s = 0
    for i in range(n):
        s += (i*i) % 97
    return s

calc()  # exécution avec mesure



## 3) Premier décorateur : **simple et utile**

### Pourquoi ?  
Pour **instrumenter** des fonctions critiques sans toucher à leur contenu. C’est idéal en **trading** pour chronométrer un calcul de signal, tracer un appel à un **broker**, ou vérifier un **contrat** d’interface.

### Comment ?  
On écrit une fonction qui prend une fonction et renvoie un **wrapper**. On y place la logique transversale (ex. timing).  
Toujours utiliser `@wraps` pour garder les **métadonnées** de la fonction d’origine.



## 4) Décorateur **paramétré**

### Pourquoi ?  
Pour **configurer** le comportement : choisir un **préfixe de log**, activer un **mode strict**, fixer un **nombre de retries**, etc.

### Comment ?  
Un décorateur paramétré ajoute **une couche** : on crée une **fabrique** de décorateurs.


In [ ]:

def with_prefix(prefix="[LOG]"):
    '''Fabrique de décorateurs : ajoute un préfixe aux traces.'''
    def deco(func):
        import functools
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(prefix, "→", func.__name__)
            return func(*args, **kwargs)
        return wrapper
    return deco

@with_prefix("[AUTH]")
def place_order(qty: int) -> str:
    '''Exemple : envoyer un ordre (jouet).'''
    return f"ordre confirmé x{qty}"

place_order(5)



## 5) Décorateurs **empilés** : comprendre l’ordre

### Idée
L’empilement permet de combiner plusieurs préoccupations : ex. **logging** puis **timing**, ou **retry** puis **timeout**.

### Ordre (à savoir par cœur)
- **Application** (lors du `import`) : du **bas vers le haut**.
- **Exécution** (à l’appel) : du **haut vers le bas** (le wrapper externe est exécuté en premier).


In [ ]:

def deco_a(f):
    def w(*a, **k):
        print("A pre"); r=f(*a, **k); print("A post"); return r
    return w

def deco_b(f):
    def w(*a, **k):
        print("B pre"); r=f(*a, **k); print("B post"); return r
    return w

@deco_a
@deco_b
def fn():
    print("FN body")

fn()
# Application: fn = deco_a(deco_b(fn))
# Exécution: A.pre -> B.pre -> body -> B.post -> A.post



## 6) Décorateur **orienté classe** : garder de l’état

### Pourquoi ?  
Quand on veut **mémoriser** des informations entre appels (compteur, contexte, cache manuel).

### Comment ?  
Une classe implémente `__call__` pour se comporter comme une fonction. L’instance devient le **wrapper**.


In [ ]:

class Counter:
    '''Décorateur de classe : compte le nombre d'appels.'''
    def __init__(self, func):
        self.func = func
        self.calls = 0
    def __call__(self, *a, **k):
        self.calls += 1
        r = self.func(*a, **k)
        print(self.func.__name__, "appel n°", self.calls)
        return r

@Counter
def ping(): 
    'Renvoie "pong".'
    return "pong"

ping(); ping()



## 7) Décorateurs **standard** à connaître

### 7.1 `lru_cache` — Mémoïsation automatique
Utile quand une fonction **pure** recalcule souvent la même chose (ex. récupération d’un **discount factor** pour le même `tenor`).  
Complexité réduite au **prix de la mémoire**.


In [ ]:

from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n: int) -> int:
    if n < 2: 
        return n
    return fib(n-1) + fib(n-2)

fib(30), fib.cache_info()



### 7.2 `singledispatch` — Polymorphisme par **type** (style fonctionnel)
Permet d’écrire une **fonction générique** et de fournir des **implémentations** selon le **type** du premier argument.


In [ ]:

from functools import singledispatch

@singledispatch
def fmt(x):
    return f"{x!r}"

@fmt.register
def _(x: int):
    return f"<int:{x}>"

@fmt.register
def _(x: float):
    return f"<float:{x:.2f}>"

fmt(42), fmt(3.14159), fmt("hello")



## 8) **Retry** avec **exponential backoff** (cas réseau/OMS)

### Contexte trading
Quand on appelle un **broker** ou un **service réseau**, on peut rencontrer des erreurs **temporaires** (latence, congestion).  
Un **retry** avec **backoff exponentiel** augmente progressivement l’attente entre les tentatives pour **soulager** le système.

### Principes
- Limiter le **nombre de tentatives**.
- **Doubler** le délai à chaque essai (`base_delay * 2**(tentative-1)`).
- Journaliser les erreurs, ne pas masquer les exceptions finales.


In [ ]:

import time, functools, random

def retry(retries=3, base_delay=0.02):
    '''Décorateur de retry simple avec backoff exponentiel.'''
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while True:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    if attempts > retries:
                        # On relance l'exception après le dernier essai
                        raise
                    delay = base_delay * (2 ** (attempts-1))
                    time.sleep(delay)
        return wrapper
    return deco

@retry(retries=3, base_delay=0.01)
def flaky_call():
    '''Simule un appel réseau instable (70% d'échec).'''
    if random.random() < 0.7:
        raise RuntimeError("temporaire")
    return "OK"

try:
    flaky_call()
except Exception as e:
    print("Toujours en échec :", e)



## 9) Bonnes pratiques, pièges, checklist d’entretien

### Bonnes pratiques
- Toujours utiliser `@functools.wraps` pour **préserver** le nom et la doc (sinon débogage pénible).
- Écrire des décorateurs **purs** (pas d’effets de bord cachés), et **petits** (lisibles).
- **Composer** des décorateurs simples plutôt qu’un décorateur “fourre‑tout”.
- Pour les signatures complexes, envisager `inspect.signature` ou la lib **`wrapt`**.

### Pièges
- **Masquer** une exception (catch silencieux) : très risqué.
- Modifier un **état global** dans un décorateur : difficile à tester.
- Empiler sans réfléchir : attention à l’**ordre d’application**.

### Checklist entretien
- Sais-tu expliquer **closure** et `wraps` ?
- Peux-tu écrire un décorateur **paramétré** proprement ?
- Sais-tu **empiler** et expliquer l’**ordre** ?
- Connais-tu `lru_cache`, `singledispatch` ?
- Sais-tu coder un **retry** raisonnable ?



## 10) Exercices guidés (avec corrigé)

### Exercice A — `retry` avec **journalisation**
**Tâche** : adapter le décorateur `retry` pour **logger** chaque tentative (numéro d’essai + exception).  
**Indice** : ajoute un `print` (ou un logger) dans le `except`.

> 👉 Corrigé ci‑dessous.


In [ ]:

import time, functools, random

def retry_logged(retries=3, base_delay=0.02):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while True:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    print(f"[retry] tentative {attempts} sur {func.__name__}: {e}")
                    if attempts > retries:
                        raise
                    time.sleep(base_delay * (2 ** (attempts-1)))
        return wrapper
    return deco

@retry_logged(retries=2, base_delay=0.01)
def flaky2():
    if random.random() < 0.8:
        raise RuntimeError("réseau saturé")
    return "OK"

try:
    flaky2()
except Exception as e:
    print("Échec final (attendu):", e)



### Exercice B — cache **manuel** simple
**Tâche** : écrire un décorateur `memo_simple` qui mémorise les résultats par **tuple(args, frozenset(kwargs.items()))`.  
**Indice** : dictionnaire interne au wrapper.

*(À faire en autonomie — compare ensuite avec `lru_cache`.)*
